In [41]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier, LogisticRegression

import xgboost as xgb
from sklearn.metrics import accuracy_score,auc,confusion_matrix,f1_score, \
    precision_score,recall_score,roc_curve
from imblearn.over_sampling import SMOTE
import prettytable
from datetime import date


In [42]:
dfoff = pd.read_csv('./data/ccf_offline_stage1_train.csv')
dftest = pd.read_csv('./data/ccf_offline_stage1_test_revised.csv')
dfon = pd.read_csv('./data/ccf_online_stage1_train.csv')

dfsub = pd.DataFrame({'User_id':'用户ID',
                      'Coupon_id':'优惠券ID',
                      'Date_received':'领取优惠券日期',
                      'Probability':'预测值'},index=[1])

# 只用线下数据，不做特征，找几个基本的模型，然后提交

## 1、预处理
正负样本标签(优惠券消费1，普通消费-1，领券不消费0)  
过采样  
训练和测试子集划分   
还是要处理一下特征，优惠力度、消费日期等    

### 1.1、打标签并且划出优惠券相关集
这个子集要怎么划分呢？题目是预测领取优惠券后是否核销，也就是说全部是1（用券消费）和0（有券不消费），不存在-1（普通消费）  
那么训练子集也要非0

In [43]:
def get_label_off(dfoff):
    """给样本打标签"""
    # 拿券，消费
    dfoff.loc[(dfoff.Coupon_id.notna()) & (dfoff.Date.notna()),'label'] = 1
    # 拿券，不消费
    dfoff.loc[(dfoff.Coupon_id.notna()) & (dfoff.Date.isna()),'label'] = 0
    #不拿券，消费
    dfoff.loc[(dfoff.Coupon_id.isna()) & (dfoff.Date.notna()),'label'] = -1
    #不拿券，不消费
    dfoff.loc[(dfoff.Coupon_id.isna()) & (dfoff.Date.isna()),'label'] = 'error'
    return dfoff

In [44]:
##打标签
#dfoff = get_label_off(dfoff)
##划分子集
#dfall = dfoff[dfoff.label!=-1]

In [45]:
#dfoff.label.value_counts()
##看起来，整体核销率就是，5%左右

### 1.2、处理特征


#### 1.2.1 Discount_rate
1. 将满xx减yy类型(`xx:yy`)的券变成折扣率 : `1 - yy/xx`，同时建立折扣券相关的特征 `discount_rate, discount_man, discount_jian, discount_type`
2. 将距离 `str` 转为 `int`
convert Discount_rate and Distance

In [46]:

def getDiscountType(row):
    if pd.isnull(row):
        return np.nan
    elif ':' in row:
        return 1
    else:
        return 0

def convertRate(row):
    """Convert discount to rate"""
    if pd.isnull(row):
        return 1.0
    elif ':' in str(row):
        rows = row.split(':')
        return 1.0 - float(rows[1])/float(rows[0])
    else:
        return float(row)

def getDiscountMan(row):
    if ':' in str(row):
        rows = row.split(':')
        return int(rows[0])
    else:
        return 0

def getDiscountJian(row):
    if ':' in str(row):
        rows = row.split(':')
        return int(rows[1])
    else:
        return 0

def processData(df):
    # convert discunt_rate
    df.loc[:,'discount_rate'] = df.loc[:,'Discount_rate'].apply(convertRate)
    df.loc[:,'discount_man'] =  df.loc[:,'Discount_rate'].apply(getDiscountMan)
    df.loc[:,'discount_jian'] = df.loc[:,'Discount_rate'].apply(getDiscountJian)
    df.loc[:,'discount_type'] = df.loc[:,'Discount_rate'].apply(getDiscountType)
    # print(df['discount_rate'].unique())
    # convert distance
    df['Distance'] = df['Distance'].fillna(-1).astype(int)
    return df

In [47]:
dfoff = processData(dfoff)

In [64]:
dftest = processData(dftest)

#### 1.2.2 领券以及核销日期

In [48]:
#date_received = dfoff['Date_received'].unique()
#date_received = sorted(date_received[pd.notnull(date_received)])

#date_buy = dfoff['Date'].unique()
#date_buy = sorted(date_buy[pd.notnull(date_buy)])
#date_buy = sorted(dfoff[dfoff['Date'].notnull()]['Date'])


In [49]:
#couponbydate = dfoff[dfoff['Date_received'].notnull()][['Date_received', 'Date']].groupby(['Date_received'], as_index=False).count()
#couponbydate.columns = ['Date_received','count']
#buybydate = dfoff[(dfoff['Date'].notnull()) & (dfoff['Date_received'].notnull())][['Date_received', 'Date']].groupby(['Date_received'], as_index=False).count()
#buybydate.columns = ['Date_received','count']

In [50]:
#这俩不是一样的吗

In [51]:
##统计不同领券日期，各自核销的订单数
#couponbydate = dfoff[dfoff['Date_received'].notnull()][['Date_received', 'Date']].groupby(['Date_received'], as_index=False).count()
#couponbydate.columns = ['Date_received','count']

In [52]:
#buybydate = dfoff[(dfoff['Date'].notnull()) & (dfoff['Date_received'].notnull())][['Date_received', 'Date']].groupby(['Date_received'], as_index=False).count()
## 优惠券消费
#buybydate.columns = ['Date_received','count']

#### 1.2.3 周末

In [53]:
def getWeekday(row):
    if row == 'nan':
        return np.nan
    else:
        return date(int(row[0:4]), int(row[4:6]), int(row[6:8])).weekday() + 1

# 领券是周几,没券就nan
dfoff['weekday'] = dfoff['Date_received'].astype(str).apply(getWeekday)
dftest['weekday'] = dftest['Date_received'].astype(str).apply(getWeekday)

# weekday_type :  周六和周日为1，其他为0
dfoff['weekday_type'] = dfoff['weekday'].apply(lambda x : 1 if x in [6,7] else 0 )
dftest['weekday_type'] = dftest['weekday'].apply(lambda x : 1 if x in [6,7] else 0 )

# change weekday to one-hot encoding 
weekdaycols = ['weekday_' + str(i) for i in range(1,8)]
tmpdf = pd.get_dummies(dfoff['weekday'].replace('nan', np.nan))
tmpdf.columns = weekdaycols
dfoff[weekdaycols] = tmpdf

tmpdf = pd.get_dummies(dftest['weekday'].replace('nan', np.nan))
tmpdf.columns = weekdaycols
dftest[weekdaycols] = tmpdf


def label(row):
    """无券-1；有券且15天内消费1；其他0（包括有券不消费、有券15天后消费）"""
    if pd.isnull(row['Date_received']):
        return -1
    if pd.notnull(row['Date']):
        td = pd.to_datetime(row['Date'], format='%Y%m%d') -  pd.to_datetime(row['Date_received'], format='%Y%m%d')
        if td <= pd.Timedelta(15, 'D'):
            return 1
    return 0
dfoff['label'] = dfoff.apply(label, axis = 1)


### 1.3 分割数据集

#### 1.3.1 过采样
1、Discount_rate无法识别（150：20）  
2、不能有nan( Distance Date_received )  
不能有nan,但是Date_received必须要保留nan  


In [54]:
dfall = dfoff[dfoff['label'] != -1].copy()
dfall.Date.fillna(0,inplace=True)

In [55]:
#过采样，注意Discount_rate和nan
model_smote = SMOTE()
dfall_X, dfall_y = model_smote.fit_sample(dfall.drop(['label','Discount_rate'],axis = 1),dfall.label)  # 输入数据并作过抽样处理
dfall = pd.concat([dfall_X,dfall_y],axis=1)

In [56]:
train = dfall[(dfall['Date_received'] < 20160516)].copy()
valid = dfall[(dfall['Date_received'] >= 20160516) & (dfall['Date_received'] <= 20160615)].copy()

### 1.4 模型以及得分

In [58]:
# 模型一
# feature
original_feature = ['discount_rate','discount_type','discount_man', 'discount_jian','Distance', 'weekday', 'weekday_type'] + weekdaycols
print("----train-----")
model = SGDClassifier(#lambda:
    loss='log',
    penalty='elasticnet',
    fit_intercept=True,
    max_iter=100,
    shuffle=True,
    alpha = 0.01,
    l1_ratio = 0.01,
    n_jobs=1,
    class_weight=None
)
model.fit(train[original_feature], train['label'])
print("---pred------")

# #### 预测以及结果评价
print(model.score(valid[original_feature], valid['label']))

----train-----
---pred------
0.7014816282781532


In [65]:
# 今天就酱，先提交

# test prediction for submission
y_test_pred = model.predict_proba(dftest[original_feature])
dftest1 = dftest[['User_id','Coupon_id','Date_received']].copy()
dftest1['label'] = y_test_pred[:,1]
dftest1.to_csv('submit1.csv', index=False, header=False)
dftest1.head()

,User_id,Coupon_id,Date_received,label
0,4129537,9983,20160712,0.612915
1,6949378,3429,20160706,0.712899
2,2166529,6928,20160727,0.057265
3,2166529,1808,20160727,0.168368
4,6172162,6500,20160708,0.406426


In [62]:
dftest

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,weekday,weekday_type,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7
0,4129537,450,9983,30:5,1.0,20160712,2,0,0,1,0,0,0,0,0
1,6949378,1300,3429,30:5,NaN,20160706,3,0,0,0,1,0,0,0,0
2,2166529,7113,6928,200:20,5.0,20160727,3,0,0,0,1,0,0,0,0
3,2166529,7113,1808,100:10,5.0,20160727,3,0,0,0,1,0,0,0,0
4,6172162,7605,6500,30:1,2.0,20160708,5,0,0,0,0,0,1,0,0
5,4005121,450,9983,30:5,0.0,20160706,3,0,0,0,1,0,0,0,0
6,4347394,450,9983,30:5,0.0,20160716,6,1,0,0,0,0,0,1,0
7,3094273,760,13602,30:5,1.0,20160727,3,0,0,0,1,0,0,0,0
8,5139970,450,9983,30:5,10.0,20160729,5,0,0,0,0,0,1,0,0
9,3237121,760,13602,30:5,1.0,20160703,7,1,0,0,0,0,0,0,1


In [59]:
pre_y = model.predict(valid[original_feature])

In [60]:
pd.Series(train.label).value_counts()

0    759172
1    749321
Name: label, dtype: int64

In [ ]:
y_test = valid.label
pre_y = model.fi
# 模型评估
# 混淆矩阵
tn, fp, fn, tp = confusion_matrix(y_test, pre_y).ravel()  # 获得混淆矩阵
confusion_matrix_table = prettytable.PrettyTable(['','prediction-0','prediction-1'])  # 创建表格实例
confusion_matrix_table.add_row(['actual-0',tp,fn])  # 增加第一行数据
confusion_matrix_table.add_row(['actual-1',fp,tn])  # 增加第二行数据
print('confusion matrix \n',confusion_matrix_table)

In [ ]:
valid[original_feature]

### 1.3 过采样
正样本比例过少，需要过采样

In [ ]:
dfall.label.value_counts()
X = dfall.iloc[:,:-1]
y = dfall.iloc[:,-1]

In [ ]:
model_smote = SMOTE()
#X,y = model_smote.fit_sample(X,y)